In [ ]:
# %%

import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import matplotlib.pyplot as plt

df = pd.read_csv("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/train.csv")
df.head()

,item_id,timestamp,target
0,H1,1750-01-01 00:00:00,605.0
1,H1,1750-01-01 01:00:00,586.0
2,H1,1750-01-01 02:00:00,586.0
3,H1,1750-01-01 03:00:00,559.0
4,H1,1750-01-01 04:00:00,511.0


In [ ]:
# %%

train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp"
)
train_data.head()

target
item_id timestamp                  
H1      1750-01-01 00:00:00   605.0
        1750-01-01 01:00:00   586.0
        1750-01-01 02:00:00   586.0
        1750-01-01 03:00:00   559.0
        1750-01-01 04:00:00   511.0

In [ ]:
# %%

predictor = TimeSeriesPredictor(
    prediction_length=48,
    target="target",
    eval_metric="MASE",
)

predictor.fit(
    train_data,
    time_limit=3600,
)

Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'AutogluonModels\ag-20240825_212339'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
GPU Count:          0
Memory Avail:       21.69 GB / 31.75 GB (68.3%)
Disk Space Avail:   768.35 GB / 952.44 GB (80.7%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 48,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 3600,
 'verbosity': 2}

Inferred time series frequency: 'h'
Provided train_data has 148060 rows, 200 time series. Median time series length is 700 (min=700, 

In [ ]:
# %%

test_data = TimeSeriesDataFrame.from_path("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/test.csv")

test_data.head()

target
item_id timestamp                  
H1      1750-01-01 00:00:00   605.0
        1750-01-01 01:00:00   586.0
        1750-01-01 02:00:00   586.0
        1750-01-01 03:00:00   559.0
        1750-01-01 04:00:00   511.0

In [ ]:
# %%

predictor.leaderboard(test_data)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.886555,-0.894366,38.715459,37.939043,1.028482,12
1,RecursiveTabular,-0.922354,-0.922730,2.397973,2.859433,233.563252,2
2,AutoARIMA,-0.993516,-1.221396,241.914892,199.935015,0.081691,8
3,SeasonalNaive,-1.022854,-1.216909,0.156384,2.460937,0.084836,1
4,PatchTST,-1.129353,-1.170438,0.413527,0.450507,159.760617,11
5,DirectTabular,-1.402350,-1.279539,0.463449,0.471387,8.744483,3
6,AutoETS,-1.805637,-1.968375,35.434520,34.157716,0.083724,7
7,DynamicOptimizedTheta,-1.875692,-2.118528,36.839961,25.440412,0.084717,6
8,TemporalFusionTransformer,-2.217661,-2.713524,0.439451,0.268103,669.415394,9
9,NPTS,-3.039193,-3.115323,4.983324,4.754296,0.089583,5


In [ ]:
# %%

finalModel = 'WeightedEnsemble'

predictions = predictor.predict(train_data, model=finalModel)
predictions.head()

mean         0.1         0.2         0.3  \
item_id timestamp                                                             
H1      1750-01-30 04:00:00  622.752447  601.557744  609.084693  614.302661   
        1750-01-30 05:00:00  555.094658  526.714915  536.734172  543.728507   
        1750-01-30 06:00:00  511.884539  478.583963  490.270816  498.486260   
        1750-01-30 07:00:00  481.572728  445.324282  458.029244  466.972849   
        1750-01-30 08:00:00  456.448604  416.113320  430.292733  440.239129   

                                    0.4         0.5         0.6         0.7  \
item_id timestamp                                                             
H1      1750-01-30 04:00:00  618.685166  622.752447  626.819728  631.202225   
        1750-01-30 05:00:00  549.620226  555.094658  560.569090  566.460818   
        1750-01-30 06:00:00  505.427262  511.884539  518.341807  525.282809   
        1750-01-30 07:00:00  474.535033  481.572728  488.610415  496.172595   
        1750-01-30 08:00:00  448.637288  456.448604  464.259916  472.658083   

                                    0.8         0.9  
item_id timestamp                                    
H1      1750-01-30 04:00:00  636.420194  643.947151  
        1750-01-30 05:00:00  573.455145  583.474409  
        1750-01-30 06:00:00  533.498261  545.185110  
        1750-01-30 07:00:00  505.116196  517.821166  
        1750-01-30 08:00:00  482.604471  496.783884